**Credits:** <a href="https://www.kaggle.com/abhishek" target="_blank">Abhishek Thakur</a> and <a href="https://www.kaggle.com/artgor" target="_blank">Andrew Lukyanenko</a>.

In [ ]:
import pandas as pd

In [ ]:
targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

targets_scored.shape

In [ ]:
targets_scored.head()

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')

train_features.shape

In [ ]:
train_features.head()

Let's take a look at the discrete levels of `cp_type`.

In [ ]:
train_features.cp_type.unique().tolist()

It is mentioned in the <a href="https://www.kaggle.com/c/lish-moa/data" target="_blank">competition page</a> that when `cp_type` is `'ctl_vehicle'`, the MoAs are always `0`. Hence, let's remove the rows in which `cp_type` is `'ctl_vehicle'`.

**Post-processing tip:** After generating the `submission` data frame, find the rows in which `cp_type` is `'ctl_vehicle'`, and set the predictions to `0`.

In [ ]:
trt_cp_idx = train_features[train_features.cp_type == 'trt_cp'].index

In [ ]:
targets_scored = targets_scored.iloc[trt_cp_idx.values.tolist()]

targets_scored.shape

The number of rows has reduced from `23814` to `21948`.

Next, let's generate our `5` folds.

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
targets_scored.loc[:, 'fold'] = -1 # Create a new column `fold` containing `-1`s.
targets_scored = targets_scored.sample(frac=1).reset_index(drop=True) # Shuffle the rows.
targets = targets_scored.drop('sig_id', axis=1).values # Extract the targets as an array.

In [ ]:
%%capture
mskf = MultilabelStratifiedKFold(n_splits=5)

In [ ]:
for fold_, (train_, valid_) in enumerate(mskf.split(X=targets_scored, y=targets)):
    targets_scored.loc[valid_, 'fold'] = fold_
    
targets_scored.to_csv('./targets_with_folds.csv', index=False)

We can now use the folds contained in `'targets_with_folds.csv'` across all our models.

To download / use the file, see the **Output** section of this notebook.